In [2]:
# Binary Customer Churn

# A marketing agency has many customers that use their service to produce ads for the
# client/customer websites. They've noticed that they have quite a bit of churn in clients.
# They basically randomly assign account managers right now, but want you to create a machine 
# learning model that will help predict which customers will churn (stop buying their service)
# so that they can correctly assign the customers most at risk to churn an account manager. 
# We will create a classification algorithm that will help classify whether or not a customer 
# churned. Then the company can test this against incoming data for future customers to predict which customers will churn 
# and assign them an account manager.

# The data is saved as customer_churn.csv. Here are the fields and their definitions:

# Name : Name of the latest contact at Company
# Age: Customer Age
# Total_Purchase: Total Ads Purchased
# Account_Manager: Binary 0=No manager, 1= Account manager assigned
# Years: Totaly Years as a customer
# Num_sites: Number of websites that use the service.
# Onboard_date: Date that the name of the latest contact was onboarded
# Location: Client HQ Address
# Company: Name of Client Company

In [3]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("Logistic").getOrCreate()

# Verify Spark session creation
print(spark)

your 131072x1 screen size is bogus. expect trouble
24/10/29 15:15:12 WARN Utils: Your hostname, 590-mxl3363mnx resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/10/29 15:15:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/29 15:15:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read.option('header','true').csv("data_logistic.csv",inferSchema=True)

In [5]:
df.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [6]:
#Loop through the first two rows of the dataset and print each row 

for record in df.head(2):
    print(record,'\n')

Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date=datetime.datetime(2013, 8, 30, 7, 0, 40), Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1) 

Row(Names='Kevin Mueller', Age=41.0, Total_Purchase=11916.22, Account_Manager=0, Years=6.5, Num_Sites=11.0, Onboard_date=datetime.datetime(2013, 8, 13, 0, 38, 46), Location='6157 Frank Gardens Suite 019 Carloshaven, RI 17756', Company='Wilson PLC', Churn=1) 



In [7]:
df.summary().show()

24/10/29 15:15:17 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         NULL|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                NULL|                NULL|0.16666666666666666|
| stddev|         NULL|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [8]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

Format for MLib

1) Import the necessary modeule for creatin VectorAssembly 

2) Create a VectorAssembler to assemble selected columns into a feature vector 

3) Transform the Dataframe using the Vector Assembler to add the 'Features' column 

4) Select the desired colummns 'Features' and 'Churn' from the transformed Dataframe 

5) Split the final dataset into training and testing sets 

In [9]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['Age','Total_Purchase','Account_Manager', 'Years','Num_Sites'],
                            outputCol='Features')


output = assembler.transform(df)

final_df = output.select('Features','Churn')

train_churn,test_churn = final_df.randomSplit([0.7,0.3])

In [10]:
output.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+--------------------+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|            Features|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+--------------------+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|[42.0,11066.8,0.0...|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|[41.0,11916.22,0....|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|[38.0,12884.75,0....|
|      Phillip White|4

Fit the Model 

1) Import the necessary modules 

2) Create a LogisticRegression model 

3) Fit the model on the training data 

4) Get the summary of the model's training 

5) Show the columns from the predictions Dataframe 

In [11]:
from pyspark.ml.classification import LogisticRegression

lr_churn = LogisticRegression(featuresCol='Features',labelCol='Churn')

fit_model = lr_churn.fit(train_churn)

training_summary =  fit_model.summary 

training_summary.predictions.show()

24/10/29 15:15:19 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


+--------------------+-----+--------------------+--------------------+----------+
|            Features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|  0.0|[5.41080897883343...|[0.99555185022897...|       0.0|
|[28.0,9090.43,1.0...|  0.0|[1.76067672737812...|[0.85329439526511...|       0.0|
|[28.0,11204.23,0....|  0.0|[2.04608836924042...|[0.88555177310052...|       0.0|
|[28.0,11245.38,0....|  0.0|[4.43799237979913...|[0.98831842795169...|       0.0|
|[29.0,5900.78,1.0...|  0.0|[4.70315613992091...|[0.99101484853456...|       0.0|
|[29.0,8688.17,1.0...|  1.0|[3.10247531571285...|[0.95699473400818...|       0.0|
|[29.0,9378.24,0.0...|  0.0|[5.47642267012038...|[0.99583316369876...|       0.0|
|[29.0,10203.18,1....|  0.0|[4.34824521118321...|[0.98723555643862...|       0.0|
|[29.0,12711.15,0....|  0.0|[6.23596956687482...|[0.99804609507502...|       0.0|
|[29.0,13240.01,

In [12]:
training_summary.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              Churn|         prediction|
+-------+-------------------+-------------------+
|  count|                608|                608|
|   mean|0.17598684210526316|0.14144736842105263|
| stddev|  0.381122524576113| 0.3487693740895026|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [13]:
# Evaluate the model on the test dataset
pre_and_label = fit_model.evaluate(test_churn)

# Show the predictions made on the test dataset
pre_and_label.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            Features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[25.0,9672.03,0.0...|    0|[5.48873923374386...|[0.99588396010275...|       0.0|
|[26.0,8787.39,1.0...|    1|[0.74872996662609...|[0.67890190230980...|       0.0|
|[26.0,8939.61,0.0...|    0|[7.37366298787875...|[0.99937282839101...|       0.0|
|[27.0,8628.8,1.0,...|    0|[6.28703144874340...|[0.99814318011662...|       0.0|
|[28.0,8670.98,0.0...|    0|[8.91701316706598...|[0.99986592983105...|       0.0|
|[28.0,11128.95,1....|    0|[4.79775059977825...|[0.99181919766736...|       0.0|
|[29.0,9617.59,0.0...|    0|[5.14047480224396...|[0.99417917129385...|       0.0|
|[29.0,11274.46,1....|    0|[5.10306995810510...|[0.99395866110186...|       0.0|
|[30.0,8677.28,1.0...|    0|[4.85655744225170...|[0.99228280688342...|       0.0|
|[30.0,10183.98,

In [14]:
pre_and_label.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              Churn|         prediction|
+-------+-------------------+-------------------+
|  count|                292|                292|
|   mean|0.14726027397260275|0.13013698630136986|
| stddev| 0.3549735395935993|0.33703168868392885|
|    min|                  0|                0.0|
|    max|                  1|                1.0|
+-------+-------------------+-------------------+



Evaluate Results 

1) Import Evaluation module 

2) Create a BinaryClassification Evaluator specifying the raw prediction and label columns 

3) Calculate the AUC by evaluating the predictions 

4) Display the calculated AUC 


In [15]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Create a BinaryClassificationEvaluator specifying the raw prediction and label columns
churn_evl = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Churn')

# Calculate the AUC by evaluating the predictions
AUC = churn_evl.evaluate(pre_and_label.predictions)

# Display the calculated AUC
AUC

0.7509573176426637